# Preamble

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# installs
!pip install gtfs-kit

     |████████████████████████████████| 62 kB 904 kB/s 
     |████████████████████████████████| 1.0 MB 16.3 MB/s 
     |████████████████████████████████| 994 kB 54.5 MB/s 
     |████████████████████████████████| 10.0 MB 3.5 MB/s 
     |████████████████████████████████| 6.3 MB 57.7 MB/s 
     |████████████████████████████████| 15.4 MB 55.7 MB/s 
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7609 sha256=f1a6bdb4f07b2e2c680186a3134cc5ce4cf5a60b97154da33a247c7ea33c3060
  Stored in directory: /root/.cache/pip/wheels/e5/99/37/e1839a5ad733e0d6abb7e0419fd913e8926ddf96408239ce01
  Created wheel for pycountry: filename=pycountry-19.8.18-py2.py3-none-any.whl size=10627377 sha256=c65f2da5c4a043adf5feec55ab824a6f2be11e3605720e247c38ff5049162acd
  Stored in directory: /root/.cache/pip/wheels/00/77/65/5f318c3c30b046b450a385ecfc690125f8a270cdb0f3f483b4
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6109 sha256=4e7e0302244d8c9a524f5077fd5c2bc523950bb7c

In [3]:
# imports
import json
import random
import requests
import datetime
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Inputs

In [4]:
# constant
key = 'cb50405b-9cce-440c-967c-09945d849519'

# variable
route_str = 'M15'

In [5]:
read_segments = f'/content/drive/My Drive/Bus Watcher Spec Project/Projects/Alex Amy + Sanket Shah/Data/Bus/Segment Data - Raw/{route_str}_2021-10-18.csv'
save_segments = f'/content/drive/My Drive/Bus Watcher Spec Project/Projects/Alex Amy + Sanket Shah/Data/Bus/Segment Data - Processed/{route_str}_2021-10-18.csv'
save_stops = f'/content/drive/My Drive/Bus Watcher Spec Project/Projects/Alex Amy + Sanket Shah/Data/Bus/Route Data/{route_str}_stops.json'

# Read

In [6]:
route = pd.read_csv(read_segments)

In [7]:
route['timestamp'] = pd.to_datetime(route['timestamp'])
route['next_stop_eta'] = pd.to_datetime(route['next_stop_eta'])

In [8]:
# sort timestamp
route = route.sort_values('timestamp')

# Clean

In [9]:
route['progress_rate'].value_counts()

normalProgress    85086
noProgress         3913
unknown            1626
Name: progress_rate, dtype: int64

In [10]:
route['progress_rate'].value_counts(normalize=True)

normalProgress    0.938880
noProgress        0.043178
unknown           0.017942
Name: progress_rate, dtype: float64

In [11]:
# only consider vehicles reporting normal progress (i.e. moving, generally). source: https://bustime.mta.info/wiki/Developers/SIRIMonitoredVehicleJourney
route = route[route['progress_rate'] == 'normalProgress']

In [12]:
route['next_stop_id'].isna().sum()

66

In [13]:
route['next_stop_id'].isna().sum() / route.shape[0]

0.0007756857767435301

In [14]:
route = route[route['next_stop_id'].isna() == False]

In [15]:
route['passenger_count'].isna().sum()

5527

In [16]:
route['passenger_count'].isna().sum() / route.shape[0]

0.06500823335685721

In [17]:
# compute max number of consecutive NaN values in passenger_count
max_consecutive_passenger_count_NaNs = {}
unique_trip_ids = set(route['unique_trip_id'])

for uuid in unique_trip_ids:
  df = route[route['unique_trip_id'] == uuid]
  passenger_counts = df['passenger_count']
  NaN_groups = passenger_counts.notna().cumsum()[passenger_counts.isna()]
  lengths_consecutive_NaN = NaN_groups.groupby(NaN_groups).agg(len)
  longest_NaN_group = lengths_consecutive_NaN.max()
  if np.isnan(longest_NaN_group):
    longest_NaN_group = 0
  max_consecutive_passenger_count_NaNs[uuid] = longest_NaN_group

In [18]:
max(max_consecutive_passenger_count_NaNs.values())

58

In [19]:
delinquent_uuids = [(max_consecutive_passenger_count_NaNs[uuid], uuid) for uuid in max_consecutive_passenger_count_NaNs if max_consecutive_passenger_count_NaNs[uuid] > 10]

In [20]:
for (longest_NaN_group, uuid) in random.sample(delinquent_uuids, 25):
  df = route[route['unique_trip_id'] == uuid]
  timestamps = list(df['timestamp'])
  start_time = timestamps[0].time()
  end_time = timestamps[-1].time()
  print(f'{start_time} --> {end_time} - {longest_NaN_group} / {df.shape[0]}')

06:23:41 --> 07:26:48 - 15 / 46
15:56:49 --> 17:32:15 - 55 / 55
17:28:22 --> 18:31:55 - 41 / 41
20:29:25 --> 21:25:24 - 39 / 39
08:00:08 --> 09:34:21 - 12 / 54
19:50:09 --> 20:58:45 - 11 / 47
14:13:45 --> 15:38:48 - 44 / 44
16:56:17 --> 18:25:49 - 44 / 44
16:02:11 --> 17:35:34 - 50 / 50
13:35:20 --> 14:50:08 - 47 / 47
20:09:11 --> 20:56:01 - 36 / 36
13:54:54 --> 14:16:30 - 12 / 12
09:51:04 --> 11:07:59 - 17 / 51
14:45:59 --> 16:13:22 - 41 / 42
17:04:19 --> 18:11:49 - 38 / 40
10:41:09 --> 12:08:43 - 45 / 45
08:53:12 --> 10:21:25 - 11 / 47
11:20:14 --> 12:32:44 - 34 / 34
21:56:34 --> 22:45:07 - 14 / 31
16:53:30 --> 18:39:07 - 54 / 55
12:34:47 --> 14:07:33 - 55 / 55
15:58:33 --> 17:08:54 - 31 / 35
11:04:36 --> 12:13:24 - 39 / 39
13:26:58 --> 14:18:16 - 28 / 28
15:41:50 --> 17:30:49 - 52 / 52


In [21]:
# remove unique_trip_ids with no passenger_count readings
for uuid in unique_trip_ids:
  df = route[route['unique_trip_id'] == uuid]
  num_not_nan =  df['passenger_count'].notna().sum()
  if num_not_nan == 0:
    route = route[route['unique_trip_id'] != uuid].copy()

In [22]:
# replace remaining NaN passenger_count values with 0
route['passenger_count'] = route['passenger_count'].fillna(0)

In [23]:
route.shape[0]

83108

# Stops

In [24]:
service_dates = sorted(list(set(route['service_date'])))

In [25]:
# new formulation: 
# strategy: collect minimal number of stops at the end of a route that do not match for each day ("delinquent stops") and subsequently drop them from the DataFrame
stops_dict = {}
delinquent_stops_dir_1 = set()
delinquent_stops_dir_0 = set()

service_date = service_dates[0]
url = 'http://bustime.mta.info/api/where/stops-for-route/MTA%20NYCT_{route_str}.json?key={key}&includePolylines=false&time={service_date}&version=2'.format(route_str=route_str.upper(), key=key, service_date=service_date)
response = requests.get(url).json()
response = response['data']['entry']['stopGroupings'][0]['stopGroups']
assert len(response) == 2
if response[0]['id'] == '1':
  stops_dict[1] = response[0]['stopIds']
  stops_dict[0] = response[1]['stopIds']
else:
  assert response[0]['id'] == '0'
  stops_dict[1] = response[1]['stopIds']
  stops_dict[0] = response[0]['stopIds']

for service_date in service_dates[1:]:
  url = 'http://bustime.mta.info/api/where/stops-for-route/MTA%20NYCT_{route_str}.json?key={key}&includePolylines=false&time={service_date}&version=2'.format(route_str=route_str.upper(), key=key, service_date=service_date)
  response = requests.get(url).json()
  response = response['data']['entry']['stopGroupings'][0]['stopGroups']
  assert len(response) == 2
  if response[0]['id'] == '1':
    response_stops_dir_1 = response[0]['stopIds']
    response_stops_dir_0 = response[1]['stopIds']
  else:
    assert response[0]['id'] == '0'
    response_stops_dir_1 = response[1]['stopIds']
    response_stops_dir_0 = response[0]['stopIds']

  for i in range(len(stops_dict[1]), 0, -1):
    if stops_dict[1][0:i] == response_stops_dir_1[0:i]:
      stops_dict[1] = response_stops_dir_1[0:i]
      delinquent_stops_dir_1.update(response_stops_dir_1[i:])
      break
  
  for i in range(len(stops_dict[0]), 0, -1):
    if stops_dict[0][0:i] == response_stops_dir_0[0:i]:
      stops_dict[0] = response_stops_dir_0[0:i]
      delinquent_stops_dir_0.update(response_stops_dir_0[i:])
      break

with open(save_stops, 'w') as f:
        json.dump(stops_dict, f)

# # original formulation: 
# # strategy: ensure that schedules match stop-for-stop each day
# # problem: no way of handling schedules that don't match stop-for-stop each day
# stops_dict = {}

# prior_service_date = service_dates[0]
# url = 'http://bustime.mta.info/api/where/stops-for-route/MTA%20NYCT_{route_str}.json?key={key}&includePolylines=false&time={service_date}&version=2'.format(route_str=route_str.upper(), key=key, service_date=prior_service_date)
# response = requests.get(url).json()
# response = response['data']['entry']['stopGroupings'][0]['stopGroups']
# assert len(response) == 2
# if response[0]['id'] == '1':
#   prev_stops_dir_1 = response[0]['stopIds']
#   prev_stops_dir_0 = response[1]['stopIds']
# else:
#   assert response[0]['id'] == '0'
#   prev_stops_dir_1 = response[1]['stopIds']
#   prev_stops_dir_0 = response[0]['stopIds']

# for service_date in service_dates[1:]:
#   url = 'http://bustime.mta.info/api/where/stops-for-route/MTA%20NYCT_{route_str}.json?key={key}&includePolylines=false&time={service_date}&version=2'.format(route_str=route_str.upper(), key=key, service_date=service_date)
#   response = requests.get(url).json()
#   response = response['data']['entry']['stopGroupings'][0]['stopGroups']
#   assert len(response) == 2
#   if response[0]['id'] == '1':
#     new_stops_dir_1 = response[0]['stopIds']
#     new_stops_dir_0 = response[1]['stopIds']
#   else:
#     assert response[0]['id'] == '0'
#     new_stops_dir_1 = response[1]['stopIds']
#     new_stops_dir_0 = response[0]['stopIds']
#   if (new_stops_dir_1 == prev_stops_dir_1) and (new_stops_dir_0 == prev_stops_dir_0):
#     prior_service_date = service_date
#     if service_date != service_dates[-1]:
#       prev_stops_dir_1 = new_stops_dir_1
#       prev_stops_dir_0 = new_stops_dir_0
#     else:
#       print('Stops match for all dates in range')
#       stops_dict[1] = new_stops_dir_1
#       stops_dict[0] = new_stops_dir_0
#       with open(save_stops, 'w') as f:
#         json.dump(stops_dict, f)
#   else:
#     if new_stops_dir_1 != prev_stops_dir_1:
#       print(f'Service change in direction 1 on {service_date}:')
#       for prev_stop_dir_1, new_stop_dir_1 in zip(prev_stops_dir_1, new_stops_dir_1):
#         print(prev_stop_dir_1, new_stop_dir_1)
#       print('\n')
#     if new_stops_dir_0 != prev_stops_dir_0:
#       print(f'Service change in direction 0 on {service_date}:')
#       for prev_stop_dir_0, new_stop_dir_0 in zip(prev_stops_dir_0, new_stops_dir_0):
#         print(prev_stop_dir_0, new_stop_dir_0)
#       print('\n')
#     break

In [26]:
delinquent_stops_dir_1

{'MTA_803002', 'MTA_904920'}

In [27]:
delinquent_stops_dir_0

set()

In [28]:
# drop delinquent stops
for delinquent_stop in delinquent_stops_dir_1:
  route = route[route['next_stop_id'] != delinquent_stop]
for delinquent_stop in delinquent_stops_dir_0:
  route = route[route['next_stop_id'] != delinquent_stop]

In [29]:
# zip direction and next_stop_id to look up position of next_stop_id along appropriate route stop sequence
route['direction-next_stop_id'] = list(zip(route['direction'], route['next_stop_id']))

# look up position of next_stop_id along appropriate route stop sequence and drop direction-next_stop_id column (not needed)
route['next_stop_id_pos'] = route['direction-next_stop_id'].apply(lambda x: stops_dict[x[0]].index(x[1]))
route = route.drop(columns='direction-next_stop_id')

# prepend None to route stop sequence to look up prior_stop_id using next_stop_id position and drop direction-stop_sequence column (not needed)
stops_dict[1].insert(0, None)
stops_dict[0].insert(0, None)
route['direction-stop_sequence'] = list(zip(route['direction'], route['next_stop_id_pos']))
route['prior_stop_id'] = route['direction-stop_sequence'].apply(lambda x: stops_dict[x[0]][x[1]])
route = route.drop(columns='direction-stop_sequence')

# Timestamps

In [30]:
# compute estimated number of seconds > 0 between stops
route['next_stop_est_sec'] = (route['next_stop_eta'] - route['timestamp']).dt.seconds

In [31]:
# check for missing values in next_stop_est_sec and replace pursuant to strategies described below...
num_missing = route['next_stop_est_sec'].isna().sum()
if num_missing > 0:
  print(f'{num_missing} missing values in next_stop_est_sec... attempting to replace with median unique_trip_id value')
  uuids = list(route['unique_trip_id'])
  segment_times = list(route['next_stop_est_sec'])
  for (i, (uuid, segment_time)) in enumerate(zip(uuids, segment_times)):
    if pd.isna(segment_time):
      replacement_value = route[route['unique_trip_id'] == uuid]['next_stop_est_sec'].median()
      if not pd.isna(replacement_value):
        segment_times[i] = replacement_value
        num_missing -= 1
  route['next_stop_est_sec'] = segment_times
if num_missing > 0:
  print(f'{num_missing} missing values in next_stop_est_sec... attempting to replace with median trip_id value')
  trips = list(route['trip_id'])
  for (i, (trip, segment_time)) in enumerate(zip(trips, segment_times)):
    if pd.isna(segment_time):
      replacement_value = route[route['trip_id'] == trip]['next_stop_est_sec'].median()
      if not pd.isna(replacement_value):
        segment_times[i] = replacement_value
        num_missing -= 1
  route['next_stop_est_sec'] = segment_times    
if num_missing > 0:
  print(f'{num_missing} missing values in next_stop_est_sec... attempting to replace with median next_stop_id value')
  segments = list(route['next_stop_id'])
  for (i, (segment, segment_time)) in enumerate(zip(segments, segment_times)):
    if pd.isna(segment_time):
      replacement_value = route[route['next_stop_id'] == segment]['next_stop_est_sec'].median()
      if not pd.isna(replacement_value):
        segment_times[i] = replacement_value
        num_missing -= 1
  route['next_stop_est_sec'] = segment_times
if num_missing > 0:
  print(f'{num_missing} missing values in next_stop_est_sec... replacing with global average')
  replacement_value = route['next_stop_est_sec'].median()
  for (i, segment_time) in enumerate(segment_times):
    if pd.isna(segment_time):
      segment_times[i] = replacement_value
      num_missing -= 1
  segment_times = list(route['next_stop_est_sec'])
assert num_missing == 0
print(f'\nsuccessfully replaced all missing values in next_stop_est_sec!')

1804 missing values in next_stop_est_sec... attempting to replace with median unique_trip_id value
17 missing values in next_stop_est_sec... attempting to replace with median trip_id value
8 missing values in next_stop_est_sec... attempting to replace with median next_stop_id value

successfully replaced all missing values in next_stop_est_sec!


# Drop Columns and Save

In [32]:
# drop unnecessary columns
keep = ['route', 'direction', 'trip_id', 'service_date', 'vehicle_id', 'timestamp', 'lat', 'lon', 'bearing', 'prior_stop_id', 'next_stop_id', 'next_stop_id_pos', 'next_stop_d_along_route', 'next_stop_est_sec', 'passenger_count']
route = route[keep]

In [33]:
# save df to drive as csv
route.to_csv(save_segments, index=False)